##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/federated/tutorials/tff_for_federated_learning_research_compression"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/federated/tutorials/tff_for_federated_learning_research_compression.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/federated/tutorials/tff_for_federated_learning_research_compression.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/federated/tutorials/tff_for_federated_learning_research_compression.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

# TFF para la investigación del aprendizaje federado: compresión de modelos y actualizaciones

**NOTA**: Esta colaboración ha sido verificada para que funcione con la [versión de lanzamiento más reciente](https://github.com/tensorflow/federated#compatibility) del paquete pip `tensorflow_federated`, pero el proyecto federado de TensorFlow aún se encuentra en una etapa de desarrollo previa al lanzamiento. Por lo tanto, es probable que no funcione en `master`.

En este tutorial, usamos el conjunto de datos [EMNIST](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/emnist) para demostrar cómo se habilitan algoritmos de compresión con pérdida para reducir el costo de comunicación en el algoritmo de promediado federado a través de la API `tff.learning`. Para conocer más detalles sobre el algoritmo de promediado federado, consulte el artículo científico [Communication-Efficient Learning of Deep Networks from Decentralized Data](https://arxiv.org/abs/1602.05629).

## Antes de empezar

Antes de empezar, ejecute lo que se encuentra a continuación, para asegurarse de que el entorno esté preparado correctamente. Si no ve un mensaje de inicio, consulte la guía de [instalación](../install.md) para obtener más instrucciones.

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated
!pip install --quiet --upgrade tensorflow-model-optimization

In [ ]:
%load_ext tensorboard

import functools

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

Verifique si TFF está funcionando.

In [ ]:
@tff.federated_computation
def hello_world():
  return 'Hello, World!'

hello_world()

b'Hello, World!'

## Preparación de los datos de entrada

En esta sección, cargamos y preprocesamos el conjunto de datos EMNIST incluido en TFF. Consulte el tutorial de [Aprendizaje federado para clasificación de imágenes](https://www.tensorflow.org/federated/tutorials/federated_learning_for_image_classification#preparing_the_input_data) para obtener más detalles sobre el conjunto de datos EMNIST.


In [ ]:
# This value only applies to EMNIST dataset, consider choosing appropriate
# values if switching to other datasets.
MAX_CLIENT_DATASET_SIZE = 418

CLIENT_EPOCHS_PER_ROUND = 1
CLIENT_BATCH_SIZE = 20
TEST_BATCH_SIZE = 500

emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data(
    only_digits=True)

def reshape_emnist_element(element):
  return (tf.expand_dims(element['pixels'], axis=-1), element['label'])

def preprocess_train_dataset(dataset):
  """Preprocessing function for the EMNIST training dataset."""
  return (dataset
          # Shuffle according to the largest client dataset
          .shuffle(buffer_size=MAX_CLIENT_DATASET_SIZE)
          # Repeat to do multiple local epochs
          .repeat(CLIENT_EPOCHS_PER_ROUND)
          # Batch to a fixed client batch size
          .batch(CLIENT_BATCH_SIZE, drop_remainder=False)
          # Preprocessing step
          .map(reshape_emnist_element))

emnist_train = emnist_train.preprocess(preprocess_train_dataset)

## Definición de un modelo

Aquí se define un modelo de keras basado en el CNN original FedAvg y luego envolvemos el modelo keras en una instancia de [tff.learning.models.VariableModel](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model) para que TFF pueda procesarlo.

Tenga en cuenta que necesitaremos una **función** que produzca un modelo en lugar de simplemente usar directamente un modelo. Además, la función **no puede** solo capturar un modelo preconstruido, sino que debe crear el modelo en el contexto en el que se llama. La razón es que TFF está diseñado para ir a dispositivos y necesita control sobre cuándo se construyen los recursos para que puedan capturarse y empaquetarse.

In [ ]:
def create_original_fedavg_cnn_model(only_digits=True):
  """The CNN model used in https://arxiv.org/abs/1602.05629."""
  data_format = 'channels_last'

  max_pool = functools.partial(
      tf.keras.layers.MaxPooling2D,
      pool_size=(2, 2),
      padding='same',
      data_format=data_format)
  conv2d = functools.partial(
      tf.keras.layers.Conv2D,
      kernel_size=5,
      padding='same',
      data_format=data_format,
      activation=tf.nn.relu)

  model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
      conv2d(filters=32),
      max_pool(),
      conv2d(filters=64),
      max_pool(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation=tf.nn.relu),
      tf.keras.layers.Dense(10 if only_digits else 62),
      tf.keras.layers.Softmax(),
  ])

  return model

# Gets the type information of the input data. TFF is a strongly typed
# functional programming framework, and needs type information about inputs to 
# the model.
input_spec = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0]).element_spec

def tff_model_fn():
  keras_model = create_original_fedavg_cnn_model()
  return tff.learning.models.from_keras_model(
      keras_model=keras_model,
      input_spec=input_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

## Cómo entrenar el modelo y generar métricas de entrenamiento

Ahora estamos listos para construir un algoritmo de promediado federado y entrenar el modelo definido en el conjunto de datos EMNIST.

En primer lugar, necesitamos crear un algoritmo de promediado federado con ayuda de la API [tff.learning.algorithms.build_weighted_fed_avg](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg).

In [ ]:
federated_averaging = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=tff_model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

A continuación, ejecutemos el algoritmo de promediado federado. La ejecución de un algoritmo de aprendizaje federado desde la perspectiva de TFF se ve así:

1. Inicialice el algoritmo y obtenga el estado inicial del servidor. El estado del servidor contiene la información necesaria para ejecutar el algoritmo. Recuerde que, como TFF es funcional, este estado incluye tanto cualquier estado optimizador que use el algoritmo (por ejemplo, términos de impulso) como los propios parámetros del modelo; estos se pasarán como argumentos y se devolverán como resultados de los cálculos de TFF.
2. Ejecute el algoritmo ronda por ronda. En cada ronda, se devolverá un nuevo estado del servidor como resultado del entrenamiento del modelo con los datos de cada cliente. Normalmente, en una ronda, ocurre lo siguiente:
    1. El servidor difunde el modelo a todos los clientes participantes.
    2. Cada cliente hace su trabajo en función del modelo y de sus propios datos.
    3. El servidor agrega todo el modelo para producir un estado de servidor que contiene un nuevo modelo.

Para obtener más detalles, consulte el tutorial de [Algoritmos federados personalizados, parte 2: implementación del promediado federado](https://www.tensorflow.org/federated/tutorials/custom_federated_algorithms_2).

Las métricas de entrenamiento se escriben en el directorio de Tensorboard para mostrarlas después del entrenamiento.

In [ ]:
def train(federated_averaging_process, num_rounds, num_clients_per_round, summary_writer):
  """Trains the federated averaging process and output metrics."""

  # Initialize the Federated Averaging algorithm to get the initial server state.
  state = federated_averaging_process.initialize()

  with summary_writer.as_default():
    for round_num in range(num_rounds):
      # Sample the clients parcitipated in this round.
      sampled_clients = np.random.choice(
          emnist_train.client_ids,
          size=num_clients_per_round,
          replace=False)
      # Create a list of `tf.Dataset` instances from the data of sampled clients.
      sampled_train_data = [
          emnist_train.create_tf_dataset_for_client(client)
          for client in sampled_clients
      ]
      # Round one round of the algorithm based on the server state and client data
      # and output the new state and metrics.
      result = federated_averaging_process.next(state, sampled_train_data)
      state = result.state
      train_metrics = result.metrics['client_work']['train']

      # Add metrics to Tensorboard.
      for name, value in train_metrics.items():
          tf.summary.scalar(name, value, step=round_num)
      summary_writer.flush()

In [ ]:
# Clean the log directory to avoid conflicts.
try:
  tf.io.gfile.rmtree('/tmp/logs/scalars')
except tf.errors.OpError as e:
  pass  # Path doesn't exist

# Set up the log directory and writer for Tensorboard.
logdir = "/tmp/logs/scalars/original/"
summary_writer = tf.summary.create_file_writer(logdir)

train(federated_averaging_process=federated_averaging, num_rounds=10,
      num_clients_per_round=10, summary_writer=summary_writer)

round  0, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.092454836), ('loss', 2.310193), ('num_examples', 941), ('num_batches', 51)]), broadcasted_bits=507.62Mibit, aggregated_bits=507.62Mibit
round  1, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.10029791), ('loss', 2.3102622), ('num_examples', 1007), ('num_batches', 55)]), broadcasted_bits=1015.24Mibit, aggregated_bits=1015.25Mibit
round  2, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.10710711), ('loss', 2.3048222), ('num_examples', 999), ('num_batches', 54)]), broadcasted_bits=1.49Gibit, aggregated_bits=1.49Gibit
round  3, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.1061061), ('loss', 2.3066027), ('num_examples', 999), ('num_batches', 55)]), broadcasted_bits=1.98Gibit, aggregated_bits=1.98Gibit
round  4, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.1287594), ('loss', 2.2999024), ('num_examples', 1064), ('num_batches', 58)]), broadcasted_bits=2.48Gibit, a

Inicie TensorBoard con el directorio de registro raíz que se especificó anteriormente para mostrar las métricas de entrenamiento. Los datos pueden tardar unos segundos en cargarse. Excepto por pérdida y precisión, también generamos la cantidad de datos difundidos y agregados. Los datos difundidos se refieren a tensores que el servidor envía a cada cliente, mientras que los datos agregados se refieren a tensores que cada cliente devuelve al servidor.

In [ ]:
#@test {"skip": true}
%tensorboard --logdir /tmp/logs/scalars/ --port=0

## Creación de una función de agregación personalizada

Ahora implementemos una función para usar algoritmos de compresión con pérdida en datos agregados. Usaremos la API de TFF para crear una función `tff.aggregators.AggregationFactory` para esto. Si bien es posible que los investigadores quieran implementar la suya propia (lo que se puede hacer a través de la API `tff.aggregators`), usaremos un método integrado para hacerlo, específicamente `tff.learning.compression_aggregator`.

Es importante tener en cuenta que este agregador no aplica compresión a todo el modelo a la vez. En lugar de ello, aplica la compresión solo a aquellas variables del modelo que sean suficientemente grandes. Generalmente, las variables pequeñas, como los sesgos, son más sensibles a la inexactitud y, al ser relativamente pequeñas, los ahorros potenciales en comunicación también lo son.

In [ ]:
compression_aggregator = tff.learning.compression_aggregator()
isinstance(compression_aggregator, tff.aggregators.WeightedAggregationFactory)

True

Como se vio anteriormente, el agregador de compresión es una fábrica de agregación *ponderada*, lo que significa que implica agregación ponderada (a diferencia de los agregadores destinados a la privacidad diferencial, que a menudo no están ponderados).

Esta fábrica de agregación se puede conectar directamente al FedAvg a través de su argumento `model_aggregator`.

In [ ]:
federated_averaging_with_compression = tff.learning.algorithms.build_weighted_fed_avg(
    tff_model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0),
    model_aggregator=compression_aggregator)

## Cómo entrenar nuevamente el modelo

A continuación, ejecutemos el nuevo algoritmo de promediado federado.

In [ ]:
logdir_for_compression = "/tmp/logs/scalars/compression/"
summary_writer_for_compression = tf.summary.create_file_writer(
    logdir_for_compression)

train(federated_averaging_process=federated_averaging_with_compression, 
      num_rounds=10,
      num_clients_per_round=10,
      summary_writer=summary_writer_for_compression)

round  0, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.087804876), ('loss', 2.3126457), ('num_examples', 1025), ('num_batches', 55)]), broadcasted_bits=507.62Mibit, aggregated_bits=146.47Mibit
round  1, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.073267326), ('loss', 2.3111901), ('num_examples', 1010), ('num_batches', 56)]), broadcasted_bits=1015.24Mibit, aggregated_bits=292.93Mibit
round  2, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.08925144), ('loss', 2.3071017), ('num_examples', 1042), ('num_batches', 57)]), broadcasted_bits=1.49Gibit, aggregated_bits=439.40Mibit
round  3, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.07985144), ('loss', 2.3061485), ('num_examples', 1077), ('num_batches', 59)]), broadcasted_bits=1.98Gibit, aggregated_bits=585.86Mibit
round  4, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.11947791), ('loss', 2.302166), ('num_examples', 996), ('num_batches', 55)]), broadcasted_bits=2.48

Vuelva a iniciar TensorBoard para comparar las métricas de entrenamiento entre dos ejecuciones.

Como puede ver, en Tensorboard hay una reducción significativa entre las curvas `original` y `compression` en los gráficos `aggregated_bits`, mientras que en los gráficos `loss` y `sparse_categorical_accuracy` las dos curvas son bastante similares.

En conclusión, implementamos un algoritmo de compresión que puede lograr un rendimiento similar al del algoritmo de promediado federado original mientras que el costo de comunicación se reduce considerablemente.

In [ ]:
#@test {"skip": true}
%tensorboard --logdir /tmp/logs/scalars/ --port=0

## Ejercicios

Para implementar un algoritmo de compresión personalizado y aplicarlo al bucle de entrenamiento, puede seguir estos pasos:

1. Implementar un nuevo algoritmo de compresión como subclase de [tff.aggregators.MeanFactory](https://www.tensorflow.org/federated/api_docs/python/tff/aggregators/MeanFactory).
2. Ejecutar un entrenamiento con el algoritmo de compresión para ver si funciona mejor que el algoritmo anterior.

Las preguntas de investigación abiertas potencialmente valiosas incluyen lo siguiente: cuantificación no uniforme, compresión sin pérdidas como la codificación Huffman y mecanismos para adaptar la compresión en función de la información de rondas de entrenamiento anteriores.

Materiales de lectura recomendados:

- [Expanding the Reach of Federated Learning by Reducing Client Resource Requirements](https://research.google/pubs/pub47774/)
- [Federated Learning: Strategies for Improving Communication Efficiency](https://research.google/pubs/pub45648/)
- *Sección 3.5 Comunicación y compresión * en [Advanced and Open Problems in Federated Learning](https://arxiv.org/abs/1912.04977)